In [1]:
print("test")

test


In [1]:
import torch
import transformers
from transformers import pipeline
from transformers import DistilBertModel, DistilBertForMaskedLM, DistilBertTokenizer
from optimum.quanto import freeze, quantize, qint8

/home/se/coding/school/exjobb/mycode/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-25 14:21:35.759271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740489695.775088   16160 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740489695.779617   16160 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-25 14:21:35.794515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in perfor

In [2]:
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-uncased", torch_dtype=torch.float16, attn_implementation="sdpa")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
p = pipeline(task="fill-mask", model=model, tokenizer=tokenizer)
p("I like eating [MASK]")

Device set to use cpu


[{'score': 0.3330078125,
  'token': 1012,
  'token_str': '.',
  'sequence': 'i like eating.'},
 {'score': 0.03900146484375,
  'token': 15960,
  'token_str': 'pork',
  'sequence': 'i like eating pork'},
 {'score': 0.033233642578125,
  'token': 10733,
  'token_str': 'pizza',
  'sequence': 'i like eating pizza'},
 {'score': 0.0299072265625,
  'token': 7967,
  'token_str': 'chocolate',
  'sequence': 'i like eating chocolate'},
 {'score': 0.0247039794921875,
  'token': 6240,
  'token_str': 'meat',
  'sequence': 'i like eating meat'}]

In [ ]:
quantize(model, weights=qint8, activations=None)


In [34]:
freeze(model)
print(model)

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): QLayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): QLinear(in_features=768, out_features=768, bias=True)
            (k_lin): QLinear(in_features=768, out_features=768, bias=True)
            (v_lin): QLinear(in_features=768, out_features=768, bias=True)
            (out_lin): QLinear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): QLayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropo

In [3]:
print(model)

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0

In [ ]:
import torch

from accelerate.test_utils.testing import get_backend

device, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)

model.to(device)

In [ ]:
for batch in train_dataloader:
    batch = {k: v.to('device') for k, v in batch.items()}
    print(model(**batch))
    break

[{'score': 0.09478759765625,
  'token': 1999,
  'token_str': 'in',
  'sequence': 'i like eating in'},
 {'score': 0.06463623046875,
  'token': 1012,
  'token_str': '.',
  'sequence': 'i like eating.'},
 {'score': 0.060516357421875,
  'token': 1010,
  'token_str': ',',
  'sequence': 'i like eating,'},
 {'score': 0.027923583984375,
  'token': 2085,
  'token_str': 'now',
  'sequence': 'i like eating now'},
 {'score': 0.0240631103515625,
  'token': 1998,
  'token_str': 'and',
  'sequence': 'i like eating and'}]